In [168]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [169]:
import scipy.stats as st
from functions import *
from graphics import *
from statistics_tests import *
import pandas as pd

df_final_demo = get_final_demo_df()

df_web_data_concat = get_web_data_df()

df_final_exp = get_final_exp_df()

df_all = get_df_all()

LIMPIEZA DE DATOS

In [170]:
df_all = df_all.drop("clnt_tenure_yr", axis=1)      # We don´t need this column because is derivative from clnt_tenure_mnth

QUITAMOS OUTLIERS DEL DF_ALL Y DEL QUE NO TIENE DUPLICADOS

In [171]:
df_all = outlier_slayer(df_all)

In [172]:
# df_all["time_diff"] = df_all.groupby(["client_id", "visit_id"])["date_time"].shift(1) - df_all["date_time"]

In [173]:
df_all["time_diff"] = abs(df_all.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()

In [174]:
df_all["repetition"] = df_all.duplicated(subset=["visit_id", "process_step"], keep="last").astype(int)          #0 = no, 1 = yes

In [175]:
df_all["last_step"] = df_all.groupby("visit_id")["process_step"].transform(max)

C:\Users\Iván\AppData\Local\Temp\ipykernel_15472\929070640.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_all["last_step"] = df_all.groupby("visit_id")["process_step"].transform(max)


In [176]:
# df_all.to_csv("data/pruebaNoche.csv")

In [177]:
df_all.head(20)

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,visitor_id,visit_id,process_step,date_time,variation,time_diff,repetition,last_step
0,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,228976764_46825473280_96584,4,2017-04-02 11:51:13,1,203.0,1,4
1,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,228976764_46825473280_96584,4,2017-04-02 11:47:50,1,65.0,1,4
2,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,228976764_46825473280_96584,4,2017-04-02 11:46:45,1,1417.0,0,4
3,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,228976764_46825473280_96584,3,2017-04-02 11:23:08,1,44.0,0,4
4,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,228976764_46825473280_96584,2,2017-04-02 11:22:24,1,46.0,0,4
5,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,228976764_46825473280_96584,1,2017-04-02 11:21:38,1,10.0,0,4
6,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,228976764_46825473280_96584,0,2017-04-02 11:21:28,1,0.0,0,4
7,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,104438405_2368283624_817211,0,2017-03-29 11:02:44,1,64.0,1,0
8,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,104438405_2368283624_817211,0,2017-03-29 11:01:40,1,117.0,1,0
9,836976,73.0,60.5,U,2.0,45105.30,6.0,9.0,427070339_1413275162,104438405_2368283624_817211,0,2017-03-29 10:59:43,1,68.0,1,0


In [178]:
count = df_all[(df_all["process_step"] == 4) & 
                (df_all["repetition"] == 0) & 
                (df_all["variation"] == 1)].shape[0]

count

14388